# Exercise: Implementing a Sudoku solver and testing for uniqueness

<b>Goal:</b> In this exercise, we implement a Sudoku solver using an integer program and Python-MIP. As an extra result, we will also see how such a solver can be used to test uniqueness of a solution to a given Sudoku puzzle.

---

## Solving a Sudoku puzzle: Modeling with integer variables

As you most likely all know, a Sudoku is puzzle where the goal is to fill in the cells of a 9x9 grid with integers from $1$ to $9$ such that
- there is precisely one number per cell,
- no row contains two equal numbers,
- no column contains two equal numbers, and
- no one of the nine 3x3 squares that the grid can be partitioned in contains two equal numbers.

An example of a Sudoku puzzle is given below.

<div style="background-color:white">
<center>
    <img src="sudoku_solver_example.png", style="padding-top: 10px;">
</center>
</div>

To solve such a Sudoku puzzle, we have to decide which numbers to assign to which cell. These decisions can be easily modelled by integral decision variables $x_{ijk}$ for $i,j,k\in\{1,\ldots,9\}$ such that

$$
x_{ijk} = \begin{cases} 1 & \text{if cell $(i,j)$ of the Sudoku contains number $k$}\\ 0 & \text{else} \end{cases}\enspace. 
$$

The question is how to set up suitable constraints that guarantee that a feasible $\{0,1\}$-point $x$ does indeed correspond to a solution of the Sudoku.


<b>Your first task:</b> Come up with linear constraints in the variables $x_{ijk}$ that model the conditions imposed on a valid Sudoku solution, i.e., make sure that any $\{0,1\}$-solution of your system corresponds to a feasible solution of a given Sudoku.

**Solution**

Each single square should have ONE entry between 1 and 9:
sum_k x_{ijk} = 1 for all i,j

Each row should have ONE entry between 1 and 9:
sum_i x_{ijk} = 1 for all j,k

Each column should have ONE entry between 1 and 9:
sum_j x_{ijk} = 1 for all i,k

Each 3x3-square should have ONE entry between 1 and 9:
sum_{i,j=l, ..., l+3} x_{ijk} = 1 for all k and l=0,3,6

Each single square should have the entry corresponding to the starting entry (if it exists)
x_{ijk} = 1, if cell (i,j) contains k (at the beginning), otherwise x_{ijk} = 0 for all i,j,k

---

## Implementing integer programs in Python-MIP

Implementing integer programs in Python-MIP is almost the same as implementing linear programs - except that you'll have to declare that you want to put integrality conditions on your variables. Check out the simple IP below.

In [3]:
#!pip install mip

In [4]:
#import mip

import numpy as np

"""
simpleProblem = mip.Model(name="Simple IP example", sense=mip.MINIMIZE)

x = simpleProblem.add_var(name="x", var_type=mip.INTEGER)

simpleProblem.objective = x

simpleProblem += x >= 4.5

simpleProblem.optimize()
"""

'\nsimpleProblem = mip.Model(name="Simple IP example", sense=mip.MINIMIZE)\n\nx = simpleProblem.add_var(name="x", var_type=mip.INTEGER)\n\nsimpleProblem.objective = x\n\nsimpleProblem += x >= 4.5\n\nsimpleProblem.optimize()\n'

As you can see, the `add_var` takes an additional (optional) argument `var_type`, which we can set to `INTEGER`, `BINARY` or `CONTINUOUS` (the latter one being the default). Thus, for the Sudoku problem above, you might want to use binary variables.

<b>Your second task:</b> Implement the constraints that you came up with in the first task in an integer program, and use it to find a solution to an input Sudoku problem. Observe that this is a pure feasibility problem, so you can use an IP with a constant objective.

To this end, you can assume that the Sudoku is given to you as a list of $81$ values, each representing a cell of the Sudoku read row by row from left to right; where a $0$ indicates an empty cell. An example is given below. Note that there also is a function to display a Sudoku that is given in the above form.

Make sure that your function returns the Sudoku in the same format as the input.

In [5]:
# Example Sudoku input and Sudoku printing

sudoku1 = [4, 0, 7, 0, 0, 0, 0, 0, 0, 
           0, 3, 5, 0, 9, 7, 4, 0, 0, 
           0, 9, 0, 0, 0, 0, 0, 0, 6, 
           0, 0, 0, 3, 0, 2, 0, 0, 0, 
           6, 0, 0, 0, 8, 0, 0, 0, 0, 
           0, 0, 0, 0, 0, 0, 5, 0, 0, 
           0, 0, 0, 4, 0, 0, 0, 1, 8, 
           0, 0, 3, 0, 2, 8, 0, 0, 4, 
           5, 0, 4, 0, 0, 0, 0, 9, 7]

def printSudoku(sudoku):
    # compact Sudoku printing function
    # taken from https://codegolf.stackexchange.com/questions/126930/
    #    draw-a-sudoku-board-using-line-drawing-characters
    q = lambda x,y:x+y+x+y+x
    r = lambda a,b,c,d,e:a+q(q(b*3,c),d)+e+"\n"
    print(((r(*"╔═╤╦╗") + q(q("║ %d │ %d │ %d "*3 + "║\n",r(*"╟─┼╫╢")), r(*"╠═╪╬╣")) +
            r(*"╚═╧╩╝")) % tuple(sudoku)).replace(*"0 "))

printSudoku(sudoku1)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 4 │   │ 7 ║   │   │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 3 │ 5 ║   │ 9 │ 7 ║ 4 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 9 │   ║   │   │   ║   │   │ 6 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║ 3 │   │ 2 ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │   │   ║   │ 8 │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║ 5 │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║ 4 │   │   ║   │ 1 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 3 ║   │ 2 │ 8 ║   │   │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │   │ 4 ║   │   │   ║   │ 9 │ 7 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [6]:
from ortools.sat.python import cp_model

def sudokuSolver(inputSudoku, othersolution_support=[]):
    # Create a CP model.
    model = cp_model.CpModel()

    # Variables
    variables = [[[model.NewBoolVar(f'x{i}{j}{k}')
                   for k in range(9)]
                  for j in range(9)]
                 for i in range(9)]

    # Constraints
    # Each cell should have one number.
    for i in range(9):
        for j in range(9):
            model.Add(sum(variables[i][j]) == 1)

    # Rows and columns constraints
    for k in range(9):
        for i in range(9):
            model.Add(sum(variables[i][j][k] for j in range(9)) == 1)
            model.Add(sum(variables[j][i][k] for j in range(9)) == 1)

    # Box constraints
    for l in [0, 3, 6]:
        for r in [0, 3, 6]:
            for k in range(9):
                model.Add(sum(variables[l + i][r + j][k] for i in range(3) for j in range(3)) == 1)

    # Fixed numbers
    for i in range(9):
        for j in range(9):
            k = inputSudoku[9*i+j] - 1
            if k >= 0:
                model.Add(variables[i][j][k] == 1)

    # Use the support for other solutions (if provided)
    for other_solution in othersolution_support:
        dot_product = []
        for i in range(9):
            for j in range(9):
                for k in range(9):
                    dot_product.append(variables[i][j][k] * other_solution[81*i+9*j+k])
        model.Add(sum(dot_product) < 81)

    model.Minimize(0)

    # Solve the CP
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # If no solution exists...
    if status != cp_model.OPTIMAL:
        return None, None

    # Otherwise...
    outputSudoku = [0] * len(inputSudoku)
    support = [0] * (9**3)

    for i in range(9):
        for j in range(9):
            for k in range(9):
                if solver.Value(variables[i][j][k]) == 1:
                    outputSudoku[9*i+j] = k + 1
                    support[81*i+9*j+k] = 1

    return outputSudoku, support

"""
# Implementation of a Sudoku solver
# TODO: sudoku3 gives a wrong solution!

def sudokuSolver(inputSudoku,othersolution_support=[0]*(9**3)):
    
    #create the IP-min problem
    sudoku_solver = mip.Model(name="Solve the Sudoku", sense=mip.MINIMIZE)
    
    #variables
    variables = [[[sudoku_solver.add_var(name=f'x{i}{j}{k}', var_type=mip.BINARY)
                    for i in range(9)]
                    for j in range(9)]
                    for k in range(9)]
    
    #objective
    var_support = [variables[i][j][k] for k in range(9) for j in range(9) for i in range(9)]
    
    sudoku_solver.objective = np.array(othersolution_support) @ var_support
    
    #constraints
    for i in range (9):
        for j in range (9):
            sudoku_solver += mip.xsum(variables[i][j][k]
                               for k in range(9)) == 1
    
    for i in range (9):
        for k in range (9):
            sudoku_solver += mip.xsum(variables[i][j][k]
                               for j in range(9)) == 1
    
    for j in range (9):
        for k in range (9):
            sudoku_solver += mip.xsum(variables[i][j][k]
                               for i in range(9)) == 1
                    
    for l in [0,3,6]:
        for r in [0,3,6]:
            for k in range (9):
                sudoku_solver += mip.xsum(variables[l+i][r+j][k]
                               for i in range(3) 
                               for j in range(3)) == 1
    
    #Add the fixed entries.
    for i in range (9):
        for j in range (9):
            k = inputSudoku[9*i+j]-1
            if k >= 0:
                sudoku_solver += variables[i][j][k] == 1
    
    #Solve the IP.
    status = sudoku_solver.optimize()
    
    #If no solution exists...
    if status != mip.OptimizationStatus.OPTIMAL:
        return None, None                       
    
    #Otherwise...
    outputSudoku = [0]*len(inputSudoku)           
    support = [0]*(9**3)
    
    for i in range (9):
        for j in range (9):
            for k in range (9):
                if variables[i][j][k].x == 1:
                    outputSudoku[9*i+j] = k+1
                    support[81*i+9*j+k] = 1
    
    return outputSudoku, support
"""

'\n# Implementation of a Sudoku solver\n# TODO: sudoku3 gives a wrong solution!\n\ndef sudokuSolver(inputSudoku,othersolution_support=[0]*(9**3)):\n    \n    #create the IP-min problem\n    sudoku_solver = mip.Model(name="Solve the Sudoku", sense=mip.MINIMIZE)\n    \n    #variables\n    variables = [[[sudoku_solver.add_var(name=f\'x{i}{j}{k}\', var_type=mip.BINARY)\n                    for i in range(9)]\n                    for j in range(9)]\n                    for k in range(9)]\n    \n    #objective\n    var_support = [variables[i][j][k] for k in range(9) for j in range(9) for i in range(9)]\n    \n    sudoku_solver.objective = np.array(othersolution_support) @ var_support\n    \n    #constraints\n    for i in range (9):\n        for j in range (9):\n            sudoku_solver += mip.xsum(variables[i][j][k]\n                               for k in range(9)) == 1\n    \n    for i in range (9):\n        for k in range (9):\n            sudoku_solver += mip.xsum(variables[i][j][k]\n  

In [7]:
l = [[0 for i in range(3)] for j in range(3)]

l[2][1] = 1

print(l)


sum(l[i][j]
                               for i in range(3) 
                               for j in range(3))


[[0, 0, 0], [0, 0, 0], [0, 1, 0]]


1

In [8]:
printSudoku(sudoku1)

solution_sudoku1, _ = sudokuSolver(sudoku1)

printSudoku(solution_sudoku1)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 4 │   │ 7 ║   │   │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 3 │ 5 ║   │ 9 │ 7 ║ 4 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 9 │   ║   │   │   ║   │   │ 6 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║ 3 │   │ 2 ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │   │   ║   │ 8 │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║ 5 │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║ 4 │   │   ║   │ 1 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 3 ║   │ 2 │ 8 ║   │   │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │   │ 4 ║   │   │   ║   │ 9 │ 7 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 4 │ 6 │ 7 ║ 2 │ 1 │ 3 ║ 9 │ 8 │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 8 │ 3 │ 5 ║ 6 │ 9 │ 7 ║ 4 │ 2 │ 1 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │ 9 │ 2 ║ 8 │ 4 │ 5 ║ 7 │ 3 │ 6 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 7 │ 5 │ 1

In [9]:
# This function creates a string representation of a given python-mip model
def model_to_str(model):
    s = f'{model.name}:\n{model.sense}\n{model.objective}\n'
    if model.constrs:
        s += 'SUBJECT TO\n' + '\n'.join(map(str, model.constrs)) + '\n'
    s += 'VARIABLES\n' + '\n'.join(f'{v.lb} <= {v.name} <= {v.ub} {v.var_type}' for v in model.vars)
    return s

# Display our LP


---

## Checking for uniqueness of the Sudoku solutions

Sudokus are generally agreed to only be "real" Sudokus if they have a unique Solution.

<b>Your third task:</b> Implement a function that checks whether a Sudoko has no solution, a unique solution, or more than one solution! You can reuse the code that you generated for the Sudoku solver above. The function should return a tuple `(n, sol)`, where $n\in\{0, 1, 2\}$ depending on whether the Sudoku has zero, one, or at least two solutions, respectively, and `sol` is a list of zero, one, or two solutions of the Sudoku.

If you want a hint, run the following code cell. Do not run it if you want to think about the problem yourself! :)

In [10]:
## Running this cell will display a hint!

encoded = [79, 98, 115, 101, 114, 118, 101, 32, 116, 104, 97, 116, 32, 115, 111, 108, 118, 105, 110, 103, 32, 97, 32, 83, 117, 100, 111, 107, 117, 32, 100, 105, 100, 32, 110, 111, 116, 32, 117, 115, 101, 32, 116, 104, 101, 32, 111, 98, 106, 101, 99, 116, 105, 118, 101, 32, 102, 117, 110, 99, 116, 105, 111, 110, 32, 111, 102, 32, 116, 104, 101, 32, 73, 80, 46, 32, 79, 110, 99, 101, 32, 121, 111, 117, 32, 102, 111, 117, 110, 100, 32, 111, 110, 101, 32, 115, 111, 108, 117, 116, 105, 111, 110, 44, 32, 99, 97, 110, 32, 121, 111, 117, 32, 101, 120, 112, 108, 111, 105, 116, 32, 116, 104, 101, 32, 102, 97, 99, 116, 32, 116, 104, 97, 116, 32, 121, 111, 117, 32, 99, 97, 110, 32, 99, 104, 111, 111, 115, 101, 32, 116, 104, 101, 32, 111, 98, 106, 101, 99, 116, 105, 118, 101, 32, 116, 111, 32, 115, 101, 101, 32, 105, 102, 32, 121, 111, 117, 32, 99, 97, 110, 32, 102, 105, 110, 100, 32, 97, 110, 111, 116, 104, 101, 114, 32, 115, 111, 108, 117, 116, 105, 111, 110, 63]
print('Hint: ' + ''.join([chr(x) for x in encoded]))

Hint: Observe that solving a Sudoku did not use the objective function of the IP. Once you found one solution, can you exploit the fact that you can choose the objective to see if you can find another solution?


In [11]:
def numberOfSolutions(inputSudoku, maxNumber = 10):
    sols = []
    supps = []
    
    sol, supp = sudokuSolver(inputSudoku)
    
    while sol != None:
        print(len(sols))
        sols.append(sol)
        supps.append(supp)
        if len(sols) == maxNumber:
            break
        sol, supp = sudokuSolver(inputSudoku, supps)
    
    ## Your code goes here.
    
    return (len(sols), sols)

---

## Testing your code

Among the following three Sudokus, there is one from each category that your function `numberOfSolutions()` should be able to distinguish: One has no solution, one has a unique Solution, and one has two Solutions. Test your implementation on these Sudokus!

In [12]:
sudoku2 = [2, 0, 0, 0, 0, 0, 0, 4, 0, 
           1, 0, 0, 0, 0, 0, 0, 0, 7,
           8, 0, 6, 3, 0, 0, 0, 0, 0,
           0, 5, 0, 0, 0, 7, 3, 0, 1, 
           0, 0, 3, 0, 1, 0, 0, 0, 0, 
           0, 0, 2, 0, 0, 3, 7, 5, 4, 
           0, 0, 7, 0, 0, 5, 0, 0, 0, 
           5, 0, 0, 0, 4, 0, 0, 0, 0, 
           0, 0, 0, 1, 7, 0, 0, 0, 8]
printSudoku(sudoku2)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 2 │   │   ║   │   │   ║   │ 4 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │   │   ║   │   │   ║   │   │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 8 │   │ 6 ║ 3 │   │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │ 5 │   ║   │   │ 7 ║ 3 │   │ 1 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 3 ║   │ 1 │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 2 ║   │   │ 3 ║ 7 │ 5 │ 4 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │ 7 ║   │   │ 5 ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │   │   ║   │ 4 │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║ 1 │ 7 │   ║   │   │ 8 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [13]:
sudoku3 = [0, 0, 0, 6, 0, 7, 0, 0, 0, 
           0, 0, 0, 0, 0, 0, 0, 9, 8,
           3, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 2, 0, 6, 0, 0, 
           0, 0, 0, 0, 0, 0, 7, 0, 0, 
           0, 4, 0, 0, 8, 0, 0, 0, 0, 
           1, 0, 0, 0, 0, 0, 0, 2, 3, 
           0, 0, 8, 9, 0, 0, 0, 0, 0, 
           0, 0, 0, 4, 0, 0, 1, 0, 0]
printSudoku(sudoku3)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║   │   │   ║ 6 │   │ 7 ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║   │ 9 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │   │   ║   │   │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │ 2 │   ║ 6 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║ 7 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 4 │   ║   │ 8 │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 1 │   │   ║   │   │   ║   │ 2 │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 8 ║ 9 │   │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║ 4 │   │   ║ 1 │   │   ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [14]:
sudoku4 = [0, 6, 0, 0, 0, 0, 0, 7, 4,
           1, 0, 0, 6, 0, 7, 0, 0, 3, 
           7, 0, 0, 0, 0, 0, 0, 0, 0, 
           0, 0, 0, 0, 1, 0, 0, 0, 2, 
           0, 0, 1, 5, 0, 0, 9, 0, 0, 
           9, 0, 0, 8, 0, 0, 0, 1, 0, 
           0, 0, 0, 0, 0, 0, 0, 3, 0, 
           3, 0, 0, 0, 0, 2, 8, 5, 0, 
           0, 0, 9, 0, 0, 4, 0, 0, 0]
printSudoku(sudoku4)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║   │ 6 │   ║   │   │   ║   │ 7 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │   │   ║ 6 │   │ 7 ║   │   │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │   │   ║   │   │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │ 1 │   ║   │   │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 1 ║ 5 │   │   ║ 9 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │   │   ║ 8 │   │   ║   │ 1 │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │   │   ║   │ 3 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │   │   ║   │   │ 2 ║ 8 │ 5 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 9 ║   │   │ 4 ║   │   │   ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [15]:
sudoku5 = [0, 0, 0, 0, 0, 0, 0, 0, 4,
           1, 0, 0, 6, 0, 7, 0, 0, 3, 
           7, 0, 0, 0, 0, 0, 0, 0, 0, 
           0, 0, 0, 0, 1, 0, 0, 0, 2, 
           0, 0, 1, 5, 0, 0, 9, 0, 0, 
           9, 0, 0, 8, 0, 0, 0, 1, 0, 
           0, 0, 0, 0, 0, 0, 0, 3, 0, 
           3, 0, 0, 0, 0, 2, 8, 5, 0, 
           0, 0, 9, 0, 0, 4, 0, 0, 0]
printSudoku(sudoku5)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║   │   │   ║   │   │   ║   │   │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │   │   ║ 6 │   │ 7 ║   │   │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │   │   ║   │   │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │ 1 │   ║   │   │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 1 ║ 5 │   │   ║ 9 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │   │   ║ 8 │   │   ║   │ 1 │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │   │   ║   │ 3 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │   │   ║   │   │ 2 ║ 8 │ 5 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 9 ║   │   │ 4 ║   │   │   ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [16]:
for sudoku in [sudoku1,sudoku2,sudoku3,sudoku4,sudoku5]:
    n, sols = numberOfSolutions(sudoku)
    
    print(n)
    
    if n==0:
        print("NO SOLUTION")
    
    for i in range(n):
        printSudoku(sols[i])

0
1
╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 4 │ 6 │ 7 ║ 2 │ 1 │ 3 ║ 9 │ 8 │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 8 │ 3 │ 5 ║ 6 │ 9 │ 7 ║ 4 │ 2 │ 1 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │ 9 │ 2 ║ 8 │ 4 │ 5 ║ 7 │ 3 │ 6 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 7 │ 5 │ 1 ║ 3 │ 6 │ 2 ║ 8 │ 4 │ 9 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │ 2 │ 9 ║ 5 │ 8 │ 4 ║ 1 │ 7 │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │ 4 │ 8 ║ 9 │ 7 │ 1 ║ 5 │ 6 │ 2 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 2 │ 7 │ 6 ║ 4 │ 5 │ 9 ║ 3 │ 1 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │ 1 │ 3 ║ 7 │ 2 │ 8 ║ 6 │ 5 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │ 8 │ 4 ║ 1 │ 3 │ 6 ║ 2 │ 9 │ 7 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝

0
NO SOLUTION
0
1
╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 4 │ 8 │ 2 ║ 6 │ 9 │ 7 ║ 3 │ 1 │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │ 6 │ 1 ║ 2 │ 5 │ 3 ║ 4 │ 9 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │ 5 │ 9 ║ 8 │ 4 │ 1 ║ 2 │ 7 │ 6 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬══